## Spark streaming test

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    subreddit = config["subreddit"]
    host = config["kafka_host"]
    spark_host = config["spark_host"]

print("imported modules")

imported modules


In [2]:
try:
    spark = SparkSession.builder.appName("reddit_" + subreddit) \
                .master("spark://{}:7077".format(spark_host)) \
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
                .getOrCreate()
                # .config("spark.eventLog.enabled", "true") \
                # .config("spark.eventLog.dir", "file:///opt/workspace/events") \
    # org.apache.hadoop:hadoop-aws:3.2.0
    spark.sparkContext.setLogLevel('WARN')
    print("created spark successfully")

except Exception as e:
    print(e)

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0e3c9a81-6f58-4ea6-85ba-40d692ee2c43;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.ap

created spark successfully


In [3]:
payload_schema = StructType([
    StructField("approved_at_utc", FloatType(), True),
    StructField("subreddit", StringType(), False),
    StructField("selftext", StringType(), False),
    StructField("author_fullname", StringType(), False),
    StructField("saved", BooleanType(), False),
    StructField("mod_reason_title", StringType(), True),
    StructField("gilded", IntegerType(), False),
    StructField("clicked", BooleanType(), False),
    StructField("title", StringType(), False),
    StructField("subreddit_name_prefixed", StringType(), False),
    StructField("hidden", BooleanType(), False),
    StructField("pwls", IntegerType(), False),
    StructField("link_flair_css_class", StringType(), False),
    StructField("downs", IntegerType(), False),
    StructField("thumbnail_height", IntegerType(), True),
    StructField("top_awarded_type", StringType(), True),
    StructField("hide_score", BooleanType(), False),
    StructField("name", StringType(), False),
    StructField("quarantine", BooleanType(), False),
    StructField("link_flair_text_color", StringType(), True),
    StructField("upvote_ratio", FloatType(), False),
    StructField("author_flair_background_color", StringType(), True),
    StructField("ups", IntegerType(), False),
    StructField("total_awards_received", IntegerType(), False),
    StructField("thumbnail_width", IntegerType(), True),
    StructField("author_flair_template_id", StringType(), True),
    StructField("is_original_content", BooleanType(), False),
    StructField("secure_media", StringType(), True),
    StructField("is_reddit_media_domain", BooleanType(), False),
    StructField("is_meta", BooleanType(), False),
    StructField("category", StringType(), True),
    StructField("link_flair_text", StringType(), True),
    StructField("can_mod_post", BooleanType(), False),
    StructField("score", IntegerType(), False),
    StructField("approved_by", StringType(), True),
    StructField("is_created_from_ads_ui", BooleanType(), False),
    StructField("author_premium", BooleanType(), False),
    StructField("thumbnail", StringType(), True),
    StructField("edited", BooleanType(), False),
    StructField("author_flair_css_class", StringType(), True),
    StructField("post_hint", StringType(), False),
    StructField("content_categories", StringType(), True),
    StructField("is_self", BooleanType(), False),
    StructField("subreddit_type", StringType(), False),
    StructField("created", FloatType(), False),
    StructField("link_flair_type", StringType(), True),
    StructField("wls", IntegerType(), False),
    StructField("removed_by_category", StringType(), True),
    StructField("banned_by", StringType(), True),
    StructField("author_flair_type", StringType(), True),
    StructField("domain", StringType(), True),
    StructField("allow_live_comments", BooleanType(), False),
    StructField("link_flair_css_class", StringType(), False),
    StructField("selftext_html", StringType(), True),
    StructField("likes", IntegerType(), True),
    StructField("suggested_sort", StringType(), True),
    StructField("banned_at_utc", FloatType(), True),
    StructField("url_overridden_by_dest", StringType(), True),
    StructField("view_count", IntegerType(), True),
    StructField("archived", BooleanType(), False),
    StructField("no_follow", BooleanType(), False),
    StructField("is_crosspostable", BooleanType(), False),
    StructField("pinned", BooleanType(), False),
    StructField("over_18", BooleanType(), False),
    StructField("media_only", BooleanType(), False),
    StructField("link_flair_template_id", StringType(), True),
    StructField("can_gild", BooleanType(), False),
    StructField("spoiler", BooleanType(), False),
    StructField("locked", BooleanType(), False),
    StructField("author_flair_text", StringType(), True),
    StructField("visited", BooleanType(), False),
    StructField("removed_by", StringType(), True),
    StructField("mod_note", StringType(), True),
    StructField("distinguished", StringType(), True),
    StructField("subreddit_id", StringType(), False),
    StructField("author_is_blocked", BooleanType(), False),
    StructField("mod_reason_by", StringType(), True),
    StructField("num_reports", IntegerType(), True),
    StructField("removal_reason", StringType(), True),
    StructField("link_flair_background_color", StringType(), True),
    StructField("id", StringType(), False),
    StructField("is_robot_indexable", BooleanType(), False),
    StructField("report_reasons", StringType(), True),
    StructField("author", StringType(), False),
    StructField("discussion_type", StringType(), True),
    StructField("num_comments", IntegerType(), False),
    StructField("send_replies", BooleanType(), False),
    StructField("whitelist_status", StringType(), False),
    StructField("contest_mode", BooleanType(), False),
    StructField("author_patreon_flair", BooleanType(), False),
    StructField("author_flair_text_color", StringType(), True),
    StructField("permalink", StringType(), False),
    StructField("parent_whitelist_status", StringType(), False),
    StructField("stickied", BooleanType(), False),
    StructField("url", StringType(), False),
    StructField("subreddit_subscribers", IntegerType(), False),
    StructField("pinned", BooleanType(), False),
    StructField("created_utc", FloatType(), False),
    StructField("num_crossposts", IntegerType(), False),
    StructField("media", StringType(), True),
    StructField("is_video", BooleanType(), False),
])

In [4]:
raw_df = spark \
            .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "{}:9092".format(host)) \
                .option("subscribe", "reddit_" + subreddit) \
                .option("startingOffsets", "latest") \
                .load() \
                .selectExpr("CAST(value AS STRING)")

stage_df = raw_df.select(from_json(col("value"), payload_schema))


22/05/10 20:46:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
try:
    stage_df.writeStream \
        .trigger(processingTime='6 seconds') \
        .outputMode("update") \
        .format("console") \
        .option("truncate", "true") \
        .start() \
        .awaitTermination()

except KeyboardInterrupt:
    print("stopped streaming.")

22/05/10 20:46:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d2a2c299-0fe4-4cf0-8ceb-483e73d88484. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/10 20:46:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/05/10 20:46:36 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0-1, groupId=spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0] Error while fetching metadata with correlation id 2 : {reddit_technology=INVALID_REPLICATION_FACTOR}
22/05/10 20:46:36 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driv

stopped streaming.


22/05/10 20:46:46 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0-1, groupId=spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0] Error while fetching metadata with correlation id 85 : {reddit_technology=INVALID_REPLICATION_FACTOR}
22/05/10 20:46:46 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0-1, groupId=spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0] Error while fetching metadata with correlation id 86 : {reddit_technology=INVALID_REPLICATION_FACTOR}
22/05/10 20:46:46 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0-1, groupId=spark-kafka-source-10e1402f-3081-43eb-be26-70aa89445855--1996546265-driver-0] Error while fetching metadata with correlation id 87 : {reddit_technology=INVALID_REPLICATION_FACTOR

In [3]:
spark.stop()
print("stopped spark successfully.")

stopped spark successfully.
